In [2]:
import requests
import pandas as pd

/var/folders/yh/tjfmc6td3jd69r72yb6mpcvh0000gq/T/ipykernel_27083/222825045.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# acquire data

In [3]:
# note that this had to strip the last two parameters from the api url in the google doc
# also upped the limit 
api_url = "https://opendata.maryland.gov/api/id/crti-ybyp.json?$select=*&$order=`:id`+ASC&$limit=3000&$offset=0 "
response = requests.get(api_url)
data = response.json()

In [4]:
data[0:2]

[{'sno': '1',
  'complaint': '232216',
  'complaint_description': 'Odor Complaint',
  'complaint_type': 'Odor',
  'recieved_date': '2024-03-01',
  'incident_date': '2024-03-01',
  'county': 'Dorchester',
  'incident_closed_date': '2024-03-01',
  'incident_status_desc': 'Incident Closed - No further action',
  'incident_zip': '21835'},
 {'sno': '2',
  'complaint': '232215',
  'complaint_description': 'Odor Complaint',
  'complaint_type': 'Odor',
  'recieved_date': '2024-03-01',
  'incident_date': '2024-03-01',
  'county': 'Dorchester',
  'incident_closed_date': '2024-03-01',
  'incident_status_desc': 'Incident Closed - No further action',
  'incident_zip': '21835'}]

In [5]:
df = pd.DataFrame(data)

In [6]:
df

,sno,complaint,complaint_description,complaint_type,recieved_date,incident_date,county,incident_closed_date,incident_status_desc,incident_zip
0,1,232216,Odor Complaint,Odor,2024-03-01,2024-03-01,Dorchester,2024-03-01,Incident Closed - No further action,21835
1,2,232215,Odor Complaint,Odor,2024-03-01,2024-03-01,Dorchester,2024-03-01,Incident Closed - No further action,21835
2,3,232214,Odor Complaint,Odor,2024-02-26,2024-02-26,Dorchester,2024-02-26,Incident Closed - No further action,21835
3,4,232205,Neighbor's fireplace is producing large amount...,"Air, Smoke",2024-02-29,2024-02-29,Carroll,NaN,Referred to Outside Agency,NaN
4,5,232204,Concern of loud boom noise in the Cheverly area,Other,2024-02-28,2024-02-28,Prince George's,2024-03-01,Incident Closed-Managed,NaN
...,...,...,...,...,...,...,...,...,...,...
1502,1503,8058,Boat company sandblasting boats and blowing wh...,Fugitive Dust/Particulate Matter,2021-01-11,2021-01-11,Queen Anne's,2022-03-08,Incident Closed - No further action,NaN
1503,1504,8057,Smoke and ash from neighbor's open burning in ...,Air,2021-01-07,2021-01-07,Anne Arundel,2021-01-07,Incident Closed-No Violation Observed,20751
1504,1505,8056,Smoke from neighbor's wood burning stove.,Air,2021-01-07,2021-01-04,Charles,NaN,Under Investigation,NaN
1505,1506,8051,Concern of materials being burned in fireplace...,Air,2021-01-04,2020-12-28,Frederick,2021-01-15,Incident Closed-No Violation Observed,21770


# clean data

# analyze data

# export data